In [8]:
from time import time
import logging
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sbn

from sklearn.datasets import fetch_lfw_people
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

print(__doc__)

In [9]:
# Display progress logs on stdout
logging.basicConfig(level=logging.INFO, format='%(asctime)s %(message)s')

# #############################################################################
# Download the data, if not already on disk and load it as numpy arrays

lfw_people = fetch_lfw_people(min_faces_per_person=70, resize=0.4)

# introspect the images arrays to find the shapes (for plotting)
n_samples, h, w = lfw_people.images.shape

# for machine learning we use the 2 data directly (as relative pixel
# positions info is ignored by this model)
X=pd.DataFrame(lfw_people.data)
n_features = X.shape[1]

# the label to predict is the id of the person
y = lfw_people.target
target_names = lfw_people.target_names
n_classes = target_names.shape[0]

print("Total dataset size:")
print("Sample size: %d" % n_samples)
print("Features: %d" % n_features)
print("Total Labels: %d" % n_classes)
print("")
print("Target Name and no of sample images:")
for i in range(len(lfw_people.target_names)):   
    print("{} has {} samples".format(lfw_people.target_names[i], (y == i).sum()))

# #############################################################################
# Plotting no of images per person from the dataset
fig, ax = plt.subplots(4, 4)
plt.subplots_adjust(wspace=0.8, hspace=0.5)

for i, axi in enumerate(ax.flat):
    axi.imshow(lfw_people.images[i], cmap='gray')
    axi.set(xticks=[], yticks=[], xlabel=lfw_people.target_names[lfw_people.target[i]])
plt.show()


In [10]:
# Checking for missing values
X.isnull().sum()

In [11]:
# Checking data type
print(X.dtypes)

In [12]:
# #############################################################################
# Split into a training set and a test set using a stratified k fold

# split into a training and testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# scaling the data
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)


In [13]:
# #############################################################################
# Compute a PCA (eigenfaces) on the face dataset (treated as unlabeled
# dataset): unsupervised feature extraction / dimensionality reduction
n_components = 150

print("Extracting the top %d eigenfaces from %d faces"
      % (n_components, X_train.shape[0]))
t0 = time()
pca = PCA(n_components=n_components, svd_solver='randomized',
          whiten=True).fit(X_train)
print("done in %0.3fs" % (time() - t0))

eigenfaces = pca.components_.reshape((n_components, h, w))

print("Projecting the input data on the eigenfaces orthonormal basis")
t0 = time()
X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)
print("done in %0.3fs" % (time() - t0))

In [14]:
###############################################################################
# Optimizing hyperparameter for MultiLayer Perceptron
# hidden_layer_sizes=(10,15)
activation = ['identity','logistic','tanh','relu']
solver = ['adam', 'sgd']
learning_rate = ['constant', 'invscaling', 'adaptive']
param_grid={'activation': activation,
            'solver':solver,
            'learning_rate': learning_rate,
            }
print(f'The hyperparamters are : {param_grid}')

print('Fitting the classifier to the training set')
print('')
t0 = time()

mlp = MLPClassifier(hidden_layer_sizes=(15,10))
mlp_Grid = GridSearchCV(estimator=mlp, param_grid=param_grid, cv=5, verbose=10, n_jobs=-1)
mlp_Grid.fit(X_train_pca, y_train)

print("done in %0.3fs" % (time() - t0))
print(f'Best estimator found by grid search: {mlp_Grid.best_params_}')


# #############################################################################
# Quantitative evaluation of the model quality on the test set
print("Predicting people's names on the test set")
t0 = time()
y_predicted_mlp=mlp_Grid.predict(X_test_pca)

# Confusion Matrix 
con_matrix_mlp=confusion_matrix(y_test, y_predicted_mlp)
%matplotlib inline
plt.figure(figsize=(10,10))
plt.title('Confusion Matrix for Multilayer Perceptron')
sbn.heatmap(con_matrix_mlp, cmap="Greens", annot=True,
            cbar_kws={"label":"Color Bar"}, fmt='d',
            xticklabels=target_names, yticklabels=target_names)
plt.xlabel('Predicted Value')
plt.ylabel('True Value')
plt.show()

print("\ndone in %0.3fs" % (time() - t0))
print(classification_report(y_test, y_predicted_mlp, target_names=target_names))

# #############################################################################
# Qualitative evaluation of the predictions using matplotlib
def plot_gallery(images, titles, h, w, n_row=3, n_col=4):
    """Helper function to plot a gallery of portraits"""
    plt.figure(figsize=(1.8 * n_col, 2.4 * n_row))
    plt.subplots_adjust(bottom=0, left=.01, right=.99, top=.90, hspace=.35)
    for i in range(n_row * n_col):
        plt.subplot(n_row, n_col, i + 1)
        plt.imshow(images[i].reshape((h, w)), cmap=plt.cm.gray)
        plt.title(titles[i], size=12)
        plt.xticks(())
        plt.yticks(())


# plot the result of the prediction on a portion of the test set
def title(y_predicted_mlp, y_test, target_names, i):
    pred_name = target_names[y_predicted_mlp[i]].rsplit(' ', 1)[-1]
    true_name = target_names[y_test[i]].rsplit(' ', 1)[-1]
    return 'predicted: %s\ntrue:      %s' % (pred_name, true_name)

prediction_titles = [title(y_predicted_mlp, y_test, target_names, i)
                     for i in range(y_predicted_mlp.shape[0])]
plot_gallery(X_test, prediction_titles, h, w)


# plot the gallery of the most significative eigenfaces

eigenface_titles = ["eigenface %d" % i for i in range(eigenfaces.shape[0])]
plot_gallery(eigenfaces, eigenface_titles, h, w)
plt.show()